<a href="https://colab.research.google.com/github/TushitPalamkar/PyTorch/blob/main/MPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import random
import json
import torch
import nltk
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [35]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
def tokenizer(sentence):
  return nltk.word_tokenize(sentence)
def stem(word):
  return stemmer.stem(word.lower())

In [36]:
def bag_of_words(tokenized_sentence,words):
  bag=np.zeros(len(words),dtype=np.float32)
  sentence_words=[stem(word) for word in tokenized_sentence]
  for indx,w in enumerate(words):
    if w in sentence_words:
      bag[indx]=1;
  return bag;

In [37]:
with open('intents.json','r') as f:
  intents=json.load(f)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'How are you', 'Hello', 'Good day'],
   'responses': ['Hello! Welcome to our course platform.',
    'Hi there! How can I assist you with your courses today?',
    'Hey! What course information are you looking for?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['Goodbye! Have a great day!',
    "See you soon! Don't hesitate to return for more course info."]},
  {'tag': 'course_info',
   'patterns': ['Tell me about your courses',
    'What courses do you offer?',
    'Can you give me details about the courses?'],
   'responses': ['We offer a variety of courses in programming, data science, and more! What specific course are you interested in?',
    'You can find courses in AI, web development, machine learning, and more. Which one interests you?']},
  {'tag': 'enrollment',
   'patterns': ['How do I enroll in a course?',
    'What is the enrollment process?',
    'Can I sign up for

In [38]:
all_words=[]
tags=[]
xy=[]
nltk.download('punkt')
for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    word=tokenizer(pattern)
    all_words.extend(word)
    xy.append((word,tag))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
ignore_words=['?','.','!']
all_words=[stem(w) for w in all_words if w not in ignore_words]
all_words=sorted(set(all_words))
tags=sorted(set(tags))

In [40]:
print(len(all_words),len(tags))

66 8


In [41]:
X_train=[]
y_train=[]
for(pattern_words,tag) in xy:
  bag=bag_of_words(pattern_words,all_words)
  X_train.append(bag)
  label=tags.index(tag)
  y_train.append(label)
X_train=np.array(X_train)
y_train=np.array(y_train)

In [42]:
class ChatBot(nn.Module):
  def __init__(self,input_size,hidden_units,output_size):
    super().__init__()
    self.layers=nn.Sequential(
        nn.Linear(in_features=input_size,out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,out_features=output_size)
    )
  def forward(self,x):
    return self.layers(x)

In [43]:
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

66 8


In [44]:
class CharDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=y_train
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  def __len__(self):
    return self.n_samples


In [45]:
dataset=CharDataset()
train_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=0)
model_0=ChatBot(input_size,hidden_size,output_size)
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model_0.parameters(),lr=learning_rate)

In [46]:
for epoch in range(num_epochs):
  for (words,labels) in train_loader:
    outputs=model_0(words)
    loss_val=loss(outputs,labels)
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()
  if (epoch+1)%100 == 0:
    print(f"Epochs:{epoch} | Loss:{loss_val.item():.4f}")
print(f"Final Loss: {loss_val.item()}")

Epochs:99 | Loss:0.6782
Epochs:199 | Loss:0.0641
Epochs:299 | Loss:0.0864
Epochs:399 | Loss:0.0046
Epochs:499 | Loss:0.0013
Epochs:599 | Loss:0.0041
Epochs:699 | Loss:0.0004
Epochs:799 | Loss:0.0044
Epochs:899 | Loss:0.0004
Epochs:999 | Loss:0.0006
Final Loss: 0.0005972169456072152


In [101]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenizer(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X=torch.tensor(X,dtype=torch.float32)

    output = model_0(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
Sam: Hello! Welcome to our course platform.
Sam: I do not understand...
Sam: You can find courses in AI, web development, machine learning, and more. Which one interests you?
Sam: To enroll, visit our website and select the course you want. You'll find an 'Enroll' button there.


tensor([6, 3, 3, 0, 1, 5, 5, 7])
tensor([0, 0, 4, 7, 1, 2, 5, 7])
tensor([5, 6, 2, 2, 6, 5, 4, 4])
tensor([3, 1])
